In [69]:
from datasets import load_dataset
train_dataset = load_dataset("imdb", split="train").to_pandas()
test_dataset = load_dataset("imdb", split="test").to_pandas()


Reusing dataset imdb (/Users/alexisjulien/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Reusing dataset imdb (/Users/alexisjulien/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


In [70]:
test_dataset

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0
...,...,...
24995,Just got around to seeing Monster Man yesterda...,1
24996,I got this as part of a competition prize. I w...,1
24997,I got Monster Man in a box set of three films ...,1
24998,"Five minutes in, i started to feel how naff th...",1


In [71]:
test_dataset[test_dataset['label'] == 0].count()

text     12500
label    12500
dtype: int64

In [72]:
train_dataset[train_dataset['label'] == 0].count()

text     12500
label    12500
dtype: int64

In [73]:
train_dataset['text'] = train_dataset['text'].str.lower()
test_dataset['text'] = test_dataset['text'].str.lower()
test_dataset

,text,label
0,i love sci-fi and am willing to put up with a ...,0
1,"worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,star rating: ***** saturday night **** friday ...,0
4,"first off let me say, if you haven't enjoyed a...",0
...,...,...
24995,just got around to seeing monster man yesterda...,1
24996,i got this as part of a competition prize. i w...,1
24997,i got monster man in a box set of three films ...,1
24998,"five minutes in, i started to feel how naff th...",1


In [74]:
from string import punctuation
train_dataset['text'] = train_dataset['text'].str.replace('[{}]'.format(punctuation), '')
test_dataset['text'] = test_dataset['text'].str.replace('[{}]'.format(punctuation), '')
test_dataset

/var/folders/np/j5wjl2kn3x94vk9x_x7_grx80000gn/T/ipykernel_93493/175751739.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_dataset['text'] = train_dataset['text'].str.replace('[{}]'.format(punctuation), '')
/var/folders/np/j5wjl2kn3x94vk9x_x7_grx80000gn/T/ipykernel_93493/175751739.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test_dataset['text'] = test_dataset['text'].str.replace('[{}]'.format(punctuation), '')


,text,label
0,i love scifi and am willing to put up with a l...,0
1,worth the entertainment value of a rental espe...,0
2,its a totally average film with a few semialri...,0
3,star rating saturday night friday night fri...,0
4,first off let me say if you havent enjoyed a v...,0
...,...,...
24995,just got around to seeing monster man yesterda...,1
24996,i got this as part of a competition prize i wa...,1
24997,i got monster man in a box set of three films ...,1
24998,five minutes in i started to feel how naff thi...,1
